# Support Vector Machine with Word2Vec (+stemming)

데이터 처리에 필요한 백그라운드를 받아온다
- os: 운영체제 인터페이스
- re: 정규 표현식 라이브러리
- pandas, numpy: 데이터 분석 및 배열 계산에 필요한 라이브러리
- nltk: 자연어 처리에 필요한 라이브러리

In [1]:
import os
import re

import pandas as pd
import numpy as np
np.seterr(divide='ignore', invalid='ignore')  # 오류가 없는 코드에 오류가 있다고 발생하는 메세지 차단

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

##  데이터 경로와 훈련/테스트 데이터 할당

input / output 데이터가 저장되는 경로를 우선 지정해주고, 어떠한 데이터가 훈련 / 테스트에 사용될지 선언해준다.
또한 후반부에 사용될 훈련/검증셋 나누는 비율(eval_split), 데이터를 섞는 횟수(random_seed)의 수를 정해준다.

In [2]:
data_in_path = './data_in/'
data_out_path = './data_out/'
train_data = 'train_data.csv'
test_data = 'test_data.csv'

train_data = pd.read_csv(data_in_path + train_data)
test_data = pd.read_csv(data_in_path + test_data)

random_seed = 10
eval_split = 0.2

## 변수 지정 + 문장을 단어로 split

In [3]:
train_clause = list(train_data['clauses'])
train_label = list(train_data['label'])

#print(train_clause[0])
#print(train_label[0])

In [4]:
sentences = []
for clauses in train_clause:
    sentences.append(clauses.split())
    
#print(sentences[0])

## split된 문장을 단어 단위로 Vectorizing(Word2Vec) 해준다. 그 후, 모델을 구축한다.

- num_features: 단어 벡터의 크기
- min_word_count: 빈도수가 적은 단어는 무시
- num_workers: 사용되는 프로세스 개수
- context: 한 단어 앞,뒤로 보고 싶은 단어 개수
- downsample: 높은 빈도의 단어가 다운샘플링 되게 하기위한 Threshold

In [73]:
num_features = 100
min_word_count = 40
num_workers = 6
context = 4
downsample = 1e-3

In [74]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',level=logging.INFO)

In [75]:
from gensim.models import word2vec

model = word2vec.Word2Vec(sentences, workers=num_workers, \
           size=num_features, min_count = min_word_count, \
            window = context, sample = downsample)

2019-11-27 15:44:41,553 : INFO : collecting all words and their counts
2019-11-27 15:44:41,554 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-11-27 15:44:41,559 : INFO : collected 2139 word types from a corpus of 24305 raw words and 1659 sentences
2019-11-27 15:44:41,560 : INFO : Loading a fresh vocabulary
2019-11-27 15:44:41,562 : INFO : effective_min_count=40 retains 131 unique words (6% of original 2139, drops 2008)
2019-11-27 15:44:41,563 : INFO : effective_min_count=40 leaves 12552 word corpus (51% of original 24305, drops 11753)
2019-11-27 15:44:41,564 : INFO : deleting the raw counts dictionary of 2139 items
2019-11-27 15:44:41,565 : INFO : sample=0.001 downsamples 131 most-common words
2019-11-27 15:44:41,566 : INFO : downsampling leaves estimated 5907 word corpus (47.1% of prior 12552)
2019-11-27 15:44:41,567 : INFO : estimated required memory for 131 words and 100 dimensions: 170300 bytes
2019-11-27 15:44:41,568 : INFO : resetting layer weight

In [76]:
model_name = '1126-stem_word2vec_model'
model.save(data_out_path+model_name)

2019-11-27 15:44:42,010 : INFO : saving Word2Vec object under ./data_out/1126-stem_word2vec_model, separately None
2019-11-27 15:44:42,011 : INFO : not storing attribute vectors_norm
2019-11-27 15:44:42,011 : INFO : not storing attribute cum_table
2019-11-27 15:44:42,014 : INFO : saved ./data_out/1126-stem_word2vec_model


In [77]:
def get_features(words, model, num_features):
    feature_vector = np.zeros((num_features),dtype=np.float64)

    num_words = 0
    index2word_set = set(model.wv.index2word)

    for w in words:
        if w in index2word_set:
            num_words += 1
            feature_vector = np.add(feature_vector, model[w])

    feature_vector = np.divide(feature_vector, num_words)
    return feature_vector

In [78]:
def get_dataset(clauses, model, num_features):
    dataset = list()

    for s in clauses:
        dataset.append(get_features(s, model, num_features))

    clauseFeatureVecs = np.stack(dataset)
    
    return clauseFeatureVecs

In [79]:
train_data_vecs = get_dataset(sentences, model, num_features)

C:\Users\hsfra\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


훈련데이터의 문장과 라벨을 훈련, 검증 셋으로 나눈다.

In [81]:
from sklearn.model_selection import train_test_split
import numpy as np

vec = train_data_vecs
lab = np.array(train_label)

vec = np.nan_to_num(vec)

vec_train, vec_eval, lab_train, lab_eval = train_test_split(vec, lab, test_size=eval_split, random_state=random_seed)

In [82]:
#print(vec_train[0])

## 서포트 벡터 머신을 import해와서 변수를 지정한 후 훈련을 시작한다.

- kernel: 커널의 종류
- C: error term값을 정하는 변수
- class_weight: 학습 데이터의 균형
- random state: 데이터 셔플 횟수
- probability: 확률 추정이 가능한지 여부

In [83]:
from sklearn import svm
clf = svm.SVC(kernel = 'linear', C = 3.0, random_state = random_seed, probability = True, class_weight = 'balanced', 
              cache_size = 300)

clf.fit(vec_train, lab_train)

SVC(C=3.0, cache_size=300, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=True, random_state=10,
    shrinking=True, tol=0.001, verbose=False)

In [85]:
print("Accuracy of training: %f" % clf.score(vec_train, lab_train))
print("Accuracy: %f" % clf.score(vec_eval, lab_eval))

Accuracy of training: 0.607385
Accuracy: 0.578313


## 여기서부터 테스트

In [86]:
test_clause = list(test_data['clauses'])

In [87]:
test_clauses = []

for clauses in test_clause:
    test_clauses.append(clauses.split())

In [88]:
test_data_vecs = get_dataset(test_clauses, model, num_features)
test_vecs = np.nan_to_num(test_data_vecs)

C:\Users\hsfra\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


In [89]:
# 위에서 만든 랜덤 포레스트 분류기를 통해 예측값을 가져온다.
result = clf.predict(test_vecs)

In [90]:
import os
# 테스트 데이터 파일에 쓰기 -

test_clauses = list(test_data['clauses'])
test_label = list(test_data['label'])

if not os.path.exists(data_out_path):
    os.makedirs(data_out_path)

# 판다스 데이터 프레임을 통해 데이터를 구성해서 output에 넣는다.
output = pd.DataFrame( data={"label": test_label ,  "predict": result} )

# 이제 csv파일로 만든다.
output.to_csv(data_out_path + "svm-w2v-predict-test-stem.csv", index=False, quoting=3 )

In [91]:
from sklearn import metrics

fpr, tpr, _ = metrics.roc_curve(lab_eval, (clf.predict_proba(vec_eval)[:, 1]))
auc = metrics.auc(fpr, tpr)

print("------------")
print("Accuracy: %f" % clf.score(test_vecs, test_data['label']))  #checking the accuracy
print("Precision: %f" % metrics.precision_score(test_data['label'], result))
print("Recall: %f" % metrics.recall_score(test_data['label'], result))
print("F1-Score: %f" % metrics.f1_score(test_data['label'], result))

------------
Accuracy: 0.589372
Precision: 0.619048
Recall: 0.719149
F1-Score: 0.665354
